# AutoTrader (Q-Learning Trader)

This project aims to create a Q-Learning that learns how to trade cryptocurrencies by analysising historical data and finding the patterns of which indicates whether the agent should `BUY`, `SELL` or `NADA` when choosing its actions.

### Data Source

The data source for this experiment will be from the US-based cryptocurrency exchange [Poloniex](http://poloniex.com/). Where there are dozens of digital assets readily available for trading.

In [1]:
# Pretty display for notebooks
%matplotlib inline


# Increase size of inline plots
from pylab import rcParams
rcParams['figure.figsize'] = 16, 8

# Load poloniex API Wrapper
from poloniex import Poloniex
from datetime import datetime, date, timedelta
import time
import pandas as pd
import numpy as np
import random

#import seaborn as sns
#np.random.seed(sum(map(ord, "aesthetics")))
#from market_env import DataSource

from IPython.display import display # Allows the use of display() for DataFrames

# Previously set random_state, so that we can have reproducibility of results
random_state = 42

# Build data into data frame
def build_dataframe(data, unit = 's'):
  df = pd.DataFrame.from_dict(data)
  df['date'] = pd.to_datetime(df['date'], unit=unit)
  df.set_index('date', inplace=True)
  df = df.convert_objects(convert_numeric=True)
  return df

def to_timestamp(dt):
  return (dt - datetime(1970, 1, 1)).total_seconds()


pol = Poloniex()
pair = 'BTC_ETH'

start_date = to_timestamp(datetime(2015, 12, 31))
#end_date = to_timestamp(datetime(2016, 12, 31))
end_date = time.time()

# Timestamp periods
SECOND = 1
MINUTE = 60 * SECOND
HOUR = MINUTE * 60
DAY = 24 * HOUR
WEEK = 7 * DAY

def scale(df):
  mean_values = df.mean(axis=0)
  std_values = df.std(axis=0)
  return (df - np.array(mean_values))/ np.array(std_values)
  

## Get data from Coin Market Cap
def coin_market_cap_data():  
  import requests
  bitcoin_data = "http://coinmarketcap.northpole.ro/history.json?coin=bitcoin&year=2016"
  r = requests.get(bitcoin_data)
  coin_mkt_cap = build_dataframe(r.json()['history'], None)
  coin_mkt_cap['price'] = [float(p['usd']) for p in coin_mkt_cap['price']]
  coin_mkt_cap['marketCap'] = [float(p['usd']) for p in coin_mkt_cap['marketCap']]
  coin_mkt_cap['volume24'] = [float(p['usd']) for p in coin_mkt_cap['volume24']]
  return coin_mkt_cap


# period = 300 # 5 minutes
# data = pol.returnChartData(pair, DAY, start_date, end_date)
# data = build_dataframe(data)
# data['Return'] = data['close'].pct_change()

# pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
# data['ClosePctl'] = data['close'].expanding(100).apply(pctrank)
# data['VolumePctl'] = data['volume'].expanding(100).apply(pctrank)

#data['close'].plot()
# scale(data).plot()

In [2]:
import gym
import pandas as pd
import numpy as np
import math
from poloniex import Poloniex

def sharpe_ratio(returns, freq=365) :
  """Given a set of returns, calculates naive (rfr=0) sharpe """
  return (np.sqrt(freq) * np.mean(returns))/np.std(returns)

def prices_to_returns(prices):
  px = pd.DataFrame(prices)
  nl = px.shift().fillna(0)
  R = ((px - nl)/nl).fillna(0).replace([np.inf, -np.inf], np.nan).dropna()
  R = np.append( R[0].values, 0)
  return R

def bbands(price, length=30, numsd=2):
    """ returns average, upper band, and lower band"""
    ave = pd.stats.moments.rolling_mean(price,length)
    sd = pd.stats.moments.rolling_std(price,length)
    upband = ave + (sd*numsd)
    dnband = ave - (sd*numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)

# df = src.data[['close']]
# df['sma'], df['upper'], df['lower'] = bbands(df[['close']])
# df.plot()

In [3]:
from trading_simulator import TradingSimulator
from poloniex_data_source import PoloniexDataSource

start_date = to_timestamp(datetime(2015, 12, 31))
#end_date = to_timestamp(datetime(2016, 12, 31))
end_date = time.time()

# Timestamp periods
SECOND = 1
MINUTE = 60 * SECOND
HOUR = MINUTE * 60
DAY = 24 * HOUR
WEEK = 7 * DAY

days = 365
period_data = {
  'pair': pair,
  'period': DAY,
  'start_date': end_date - days * DAY,
  'end_date': end_date,
  'days': days
}

data_source = PoloniexDataSource(period_data)
prices = data_source.get_prices()
simulator = TradingSimulator(1.0, prices, period_data)


poloniex_data_source.py:45: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df =  df.convert_objects(convert_numeric=True)
poloniex_data_source.py:28: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df =  df.convert_objects(convert_numeric=True)
poloniex_data_source.py:31: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=7,center=False).mean()
  rolling_mean = pd.stats.moments.rolling_mean(df["close"], 7)


In [4]:
from market_env import MarketEnv
env = MarketEnv(simulator, data_source)

In [5]:
def run_sim():
  i = 0
  while i <= 5:
    env.step("BUY")
    if i == 5:
      env.step("SELL")    
    i += 1
run_sim()
simulator.holdings

,ETH,BTC
date,,
2016-08-07,54.487005,0.000000
2016-08-08,54.487005,0.000000
2016-08-09,54.487005,0.000000
2016-08-10,54.487005,0.000000
2016-08-11,54.487005,0.000000
2016-08-12,54.487005,0.000000
2016-08-13,0.000000,1.078461
2016-08-14,0.000000,0.000000
2016-08-15,0.000000,0.000000


### Visualize data

There are a few important public datapoints that are important for us in this context:

- We need to know the state of the price at that moment, given a pair. This can be accomplished by calling the `returnTicker` function passing in the trading pair (`BTC_ETH`) as an example.
- Also, it is important to have a feel for what is the state of current orders in the market, what does the current order book for this pair look like? What is the volume being traded?
- Thirdly, its important to have a feel for the price history of that given pair and how its been evolving. What is the daily return? What is the sharpe-ratio? Simple moving average?

### Environment

In order to be able to create a trading bot, we need to be able to simulate a trading environment, where the bot can visualize the current state of the market and take actions depending on it. For this to be done we need:

- An oracle (datasource) responsible for gathering data from the Exchanges client library;
- A Trading simulator that can run the trials of each step the agent needs when choosing an action, this simulator should
- A market environment where the reinforcement learning agent can act upon and measure its performance;
- A reward scheme, where the agent can get rewarded depending on its actions;
- Plot the price history over the trading period.
- Sharpe ratio (Always assume you have 365 trading days in an year. And risk free rate = 0) of the total portfolio
- Cumulative return of the total portfolio
- Standard deviation of daily returns of the total portfolio
- Average daily return of the total portfolio
- Ending value of the portfolio

#### Oracle

An oracle is a class responsible for gathering data from the market, it should:

- Get data fromt the market;
- Transform this data into desirable values. For example, daily return, bolinger value, simple moving average.
- Serve the new data everytime a step is updated;
- Reset the data frame accordingly when the step is over and the `reset()` method is called;

#### Simulator

A trading simulator responsible for simulating the act of trading on the exchange. It should:

- Keep track of the value currently invested in the coin;
- Keep track of the costs incurred with trading in the exchange;
- Compute the reward for user and return the information of this user gathered from the exchange and current state of the market.
- Compute the change in state executed on your own portfolio.
    - What happens when a `buy` order comes in?
        - We should reduce the amount of cash currently available on the account and increase the amount.
    - What happens when a `sell` order comes in?
       - Increase the amount of cash we have and reduce the amount of coins.

#### Exchange

This class is responsible for gathering information about the exchange and the cost incurred with each trade. Also, it should be responsible for keeping track of which positions we currently hold with the exchange. 

- `__init__(trader_id)`
- Build the cost table for the exchange. [See here](https://poloniex.com/fees/)
- `current_cost()` for the user
- `open_trades()`: get open trades method
- `get_balance()`: query this trader's balance

#### Portfolio 

This class is responsible for keeping track of the current trader's own portfolio. It should have:

- `get_portfolio_trades()`
- `buy()` - What happens when a `buy` order comes in? - We should deduce the amount of cash currently available on the account and increase the amount 
- `sell()` - What happens when a `sell` order comes in?

#### Define state space

In order to have a reinforcement learning agent, it is necessary to define the agents state space.

- Do we currently hold any coins or not (of whatever cryptocurrency we are currently trading), this is binary true or false (0 or 1).
- Bolinger value, how is the current state of prices going compared with the simple moving average. This value needs to be discretized, in order to be able to use it as a state. See OpenAI Gym's `Discretize` class. 
- Coin's daily return is also a good value as part of the state. This also needs to be discretized.
- Ration between the close price and the rolling mean (simple moving average)

When what we are trading is bitcoin:
- Number of transactions happening on the blockchain that date; See https://api.blockchain.info/charts/n-transactions?format=json&timespan=all
- Number of unique addresses generated. See: https://blockchain.info/charts/n-unique-addresses
- Number of unspent transaction output. Which basically means the number of unspent bitcoins being generated everyday cumulatively. See: https://blockchain.info/charts/utxo-count
- Estimated transaction value. See: https://blockchain.info/charts/estimated-transaction-volume-usd


#### Actions

- BUY (1)
- SELL (-1)
- NADA (0)

#### Agent

Agent timeline:
1. Aiming to simplify the task at hand, we shall create an agent that chooses the available actions randomly. `RandomAgent`
2. Create a Q-learning agent that trains on 2016 data and tests on 2017. `QAgent`
3. Improve Q-learning agent by introducing the concept of halucination, allowing the agent to halucinate on every single day many possible scenarios for the asset on that day. `QAgent`
